Initial setting:
- imports packages: pandas for sheets, sklearn for MI score, math for log, re for regex search
- picks up csv from Google sheets, cleans it, and converts table values to variables
- defines test function from original notebook

In [ ]:
import pandas as pd
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
import math, re
from itertools import combinations, permutations
import math

sheet_id = "1cB2M-EWl2xwBToBXg9GkEsd2w6tw_coCFkXXtV8Sqmc"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"
df = pd.read_csv(url)
df.pop("idx");

signals, categories = df[['msg', 'cat']].to_numpy().T
signals = ["".join([f"/{i}/" for i in signal.split()]) for signal in signals]

def format_n(x, n=1):
  formatted_value = f"%.{math.ceil(-math.log(x, 10)) + n - 1}f" % x
  return formatted_value

from collections import Counter
# categories: list[?]
def get_entropy(categories):
  counter = Counter(categories)
  total = counter.total()
  return entropy([(counter[x] / total) for x in counter])

# In all cases below, `feature_f` is a func[string, bool].
def test_feature(categories, signals=None, feature_f=None, features=None, normalized=False):
  assert (feature_f is None) or (features is None) # Either provide a feature function or directly provide features.
  assert (feature_f is not None) or (features is not None) # Do not both provide a feature function and directly provide features.
  assert (feature_f is None) == (signals is None) # Provide signals if and only if you provide a feature function.

  if(features is None): features = [feature_f(signal) for signal in signals]

  #print(features, categories)
  #print(normalized, mutual_info_score(features, categories), get_entropy(features))
  if(normalized):
    e = get_entropy(features)
    if(e > 0.0): return mutual_info_score(features, categories) / e # output in [0, 1], proposition of the entropy of the features.
    else: return -1
  else: return mutual_info_score(features, categories) / math.log(2) # output in bits

elements = ['/1/', '/2/', '/4/', '/5/', '/6/', '/9/', '/10/', '/11/', '/12/', '/13/', '/14/', '/15/']
elements_parentheses = ['(/1/)', '(/2/)', '(/4/)', '(/5/)', '(/6/)', '(/9/)', '(/10/)', '(/11/)', '(/12/)', '(/13/)', '(/14/)', '(/15/)']
multiples = ['{1}', '{2}', '{3}', '{4}', '{5}', '{6}', '{7}', '{8}']

In [ ]:
mi = test_feature(feature_f=(lambda signal, pat='(/4/){6}': re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
print(mi)

0.8760351920123253


In [ ]:
mi = test_feature(feature_f=(lambda signal, pat='/12/|/3/': re.search(pat,signal) is not None), signals=signals, categories=categories)
print(mi)

0.908618915835874


In [ ]:
mi = test_feature(feature_f=(lambda signal, pat=r'(?=.*/12/)(?=.*/1/).*': re.search(pat,signal) is not None), signals=signals, categories=categories)
print(mi)

0.6463557316915962


In [ ]:
mi = test_feature(feature_f=(lambda signal, pat='/12/+/5/': re.search(pat,signal) is not None), signals=signals, categories=categories)
print(mi)

0.2989886452744029


In [ ]:
mi = test_feature(feature_f=(lambda signal, pat='(/12//1/){4}': re.search(pat,signal) is not None), signals=signals, categories=categories)
print(mi)

0.000606597373548794


### Conjunctions, disjunctions (whole set)

In [ ]:
# regex disjunction: |
# regex conjunction: assert that pattern is in the same string as another with (?=pattern)
#                    r'(?=.*a)(?=.*b).*' a string that contains a and b in any order
#                    r'^.*a.*b.*$'       a string that contains a followed by b

In [ ]:
def test_all_unordered_conjunctions(elements, max_drop=None, verbose=False):
  '''
  TODO
  '''
  output = []
  for ele, els in combinations(elements, 2):
      if ele == els:
        continue
      else:
        pattern = r'(?=.*' + re.escape(ele) + r')(?=.*' + re.escape(els) + r').*'
        mi_both = test_feature(feature_f=(lambda signal, pat=pattern: re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
        mi_first = test_feature(feature_f=(lambda signal, pat=ele: re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
        mi_second = test_feature(feature_f=(lambda signal, pat=els: re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
        if verbose:
          print(f'Checking conjunction of {ele} ({format_n(mi_first,n=4)}) and {els} ({format_n(mi_second,n=4)}): {format_n(mi_both,n=4) if mi_both > 0 else mi_both}')
        if mi_both > mi_first and mi_both > mi_second:
          output.append(f'for {ele}&{els} mi is {format_n(mi_both,n=4)}, larger than {ele}: {format_n(mi_first,4)} or {els}: {format_n(mi_second,4)}')
          if verbose:
            print('Success')
        elif mi_both == mi_first and mi_first > mi_second:
          output.append(f'for {ele}&{els} mi is equal to just {ele}, which is {mi_first:.4f}')
        elif mi_both == mi_second and mi_second > mi_first:
          output.append(f'for {ele}&{els} mi is equal to just {els}, which is {mi_second:.4f}')
        elif max_drop is not None and (mi_both > (mi_first - max_drop) or mi_both > (mi_second - max_drop)):
          output.append(f'for {ele}&{els} mi is high ({format_n(mi_both,n=4)}) considering {ele}: {format_n(mi_first,n=4)} and {els}: {format_n(mi_second,n=4)}')
        else:
          continue
  return output

In [ ]:
test_all_unordered_conjunctions(elements, max_drop=0.01, verbose=False)

['for /1/&/4/ mi is 0.9255, larger than /1/: 0.8286 or /4/: 0.8592',
 'for /1/&/9/ mi is high (0.8521) considering /1/: 0.8286 and /9/: 0.9143',
 'for /1/&/12/ mi is high (0.8632) considering /1/: 0.8286 and /12/: 0.9128',
 'for /1/&/13/ mi is high (0.4309) considering /1/: 0.8286 and /13/: 0.4309',
 'for /1/&/14/ mi is high (0.4651) considering /1/: 0.8286 and /14/: 0.4297',
 'for /1/&/15/ mi is high (0.3441) considering /1/: 0.8286 and /15/: 0.2971',
 'for /2/&/4/ mi is high (0.4679) considering /2/: 0.4679 and /4/: 0.8592',
 'for /2/&/9/ mi is high (0.4679) considering /2/: 0.4679 and /9/: 0.9143',
 'for /4/&/6/ mi is high (0.6553) considering /4/: 0.8592 and /6/: 0.5378',
 'for /4/&/9/ mi is high (0.8522) considering /4/: 0.8592 and /9/: 0.9143',
 'for /4/&/14/ mi is high (0.4837) considering /4/: 0.8592 and /14/: 0.4297',
 'for /5/&/12/ mi is high (0.8045) considering /5/: 0.8045 and /12/: 0.9128',
 'for /6/&/9/ mi is high (0.6553) considering /6/: 0.5378 and /9/: 0.9143',
 'for /

In [ ]:
def test_all_disjunctions(elements, max_drop=None, verbose=False):
  output = []
  for ele, els in combinations(elements, 2):
    if ele == els:
      continue
    else:
      pattern = ele + '|' + els
      mi_both = test_feature(feature_f=(lambda signal, pat=pattern: re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
      mi_first = test_feature(feature_f=(lambda signal, pat=ele: re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
      mi_second = test_feature(feature_f=(lambda signal, pat=els: re.search(pat,signal) is not None), signals=signals, categories=categories, normalized=True)
      if verbose:
        print(f'Checking disjunction of {ele} ({format_n(mi_first,n=4)}) and {els} ({format_n(mi_second,n=4)}): {format_n(mi_both,n=4) if mi_both > 0 else mi_both}')
      if mi_both > mi_first and mi_both > mi_second:
        output.append(f'for {ele}|{els} mi is {format_n(mi_both,n=4)}, larger than {ele}: {format_n(mi_first,4)} or {els}: {format_n(mi_second,4)}')
        if verbose:
          print('Success')
      elif mi_both == mi_first and mi_first > mi_second:
        output.append(f'for {ele}|{els} mi is equal to just {ele}, which is {mi_first:.4f}')
      elif mi_both == mi_second and mi_second > mi_first:
        output.append(f'for {ele}|{els} mi is equal to just {els}, which is {mi_second:.4f}')
      elif max_drop is not None and (mi_both > (mi_first - max_drop) or mi_both > (mi_second - max_drop)):
        output.append(f'for {ele}|{els} mi is high ({format_n(mi_both,n=4)}) considering {ele}: {format_n(mi_first,n=4)} and {els}: {format_n(mi_second,n=4)}')
      else:
        continue
  return output

In [ ]:
test_all_disjunctions(elements, max_drop=0.05)

['for /1/|/2/ mi is 0.8289, larger than /1/: 0.8286 or /2/: 0.4679',
 'for /1/|/4/ mi is high (0.8330) considering /1/: 0.8286 and /4/: 0.8592',
 'for /1/|/5/ mi is high (0.8064) considering /1/: 0.8286 and /5/: 0.8045',
 'for /1/|/6/ mi is 0.8394, larger than /1/: 0.8286 or /6/: 0.5378',
 'for /1/|/9/ mi is high (0.8423) considering /1/: 0.8286 and /9/: 0.9143',
 'for /1/|/10/ mi is high (0.8263) considering /1/: 0.8286 and /10/: 0.4405',
 'for /1/|/11/ mi is 0.8291, larger than /1/: 0.8286 or /11/: 0.6238',
 'for /1/|/12/ mi is high (0.8117) considering /1/: 0.8286 and /12/: 0.9128',
 'for /1/|/13/ mi is equal to just /1/, which is 0.8286',
 'for /1/|/14/ mi is high (0.8112) considering /1/: 0.8286 and /14/: 0.4297',
 'for /1/|/15/ mi is high (0.8283) considering /1/: 0.8286 and /15/: 0.2971',
 'for /2/|/4/ mi is equal to just /4/, which is 0.8592',
 'for /2/|/5/ mi is high (0.7854) considering /2/: 0.4679 and /5/: 0.8045',
 'for /2/|/6/ mi is 0.5637, larger than /2/: 0.4679 or /6/: 

### Semi-Automatic Gram Search

[TODO] begs to be a class with `__str__` and `__repr__`

In [ ]:
def tester(pattern, signals=signals, categories=categories, normalized=True):
  return test_feature(
        feature_f=lambda signal, pat=pattern: re.search(pat, signal) is not None,
        signals=signals,
        categories=categories,
        normalized=normalized
    )

def pattern_maker():
  pass

def test_ngram(word, normalized=True):
  pattern = word # e.g. "A", "AB", "ABC"
  return tester(pattern, normalized=normalized)

def test_loopy_unigram(letter, normalized=True):
  # "AA*" matches an initial letter followed by zero or more of the same.
  pattern = f"{letter}({letter})*" # e.g. "AA*"
  return tester(pattern, normalized=normalized)

def test_bi_loopy_unigram(first_letter, second_letter, normalized=True):
  # "AA*BB*" matches a run of first_letter followed by a run of second_letter.
  pattern = f"{first_letter}({first_letter})*{second_letter}({second_letter})*" # e.g. "AA*BB*"
  return tester(pattern, normalized=normalized)

def test_loopy_bigram(word, normalized=True):
  # "AB(AB)*" matches a bigram followed by zero or more repetitions of it.
  pattern = word + "(" + word + ")*" # e.g. "AB(AB)*"
  return tester(pattern, normalized=normalized)

In [ ]:
# Unigrams: A
# Loopy unigrams: AA*
for ele in elements:
  assert test_ngram(ele) is not None
  assert test_loopy_unigram(ele) is not None

# Bigrams: AB
# Loopy bigrams: AB(AB)*
for ele, els in combinations(elements,2):
  assert test_ngram(ele + els, normalized=True) is not None
  assert test_loopy_bigram(ele + els) is not None

# Trigrams: ABC
for ele, els, elm in combinations(elements,3):
  assert test_ngram(ele + els + elm, normalized=False) is not None

# "Bi-loopy" unigrams: AA*BB*
for ele, els in permutations(elements,2):
  assert test_bi_loopy_unigram(ele, els) is not None

In [ ]:
for ele in elements:
  result = test_loopy_unigram(ele)
  if result != -1:
    print(ele + ele + '*', test_loopy_unigram(ele))

/1//1/*
/1//1/*
/1//1/* 0.8542424723130415
/2//2/*
/4//4/*
/4//4/*
/4//4/* 0.8405752061940837
/5//5/*
/5//5/*
/5//5/* 0.614839315964898
/6//6/*
/6//6/*
/6//6/* 0.3499693026649984
/9//9/*
/9//9/*
/9//9/* 0.8696683874132017
/10//10/*
/11//11/*
/11//11/*
/11//11/* 0.4807277340731558
/12//12/*
/12//12/*
/12//12/* 0.8804740082948771
/13//13/*
/14//14/*
/14//14/*
/14//14/* 0.5163534030957602
/15//15/*


In [ ]:
print(test_ngram('/1/', normalized=True))
print(test_loopy_unigram('/1/', normalized=True))

/1/
0.8286055375651996
/1/(/1/)*
0.8286055375651996


### Pattern search

Pattern search: <br>
We want to look for informative patterns within the messages. The more a pattern corresponds to a characteristic within a category, the higher MI it will yield. <br>
We assert the following: <br>
- maximum repetition length is 8 (from initial "brute force" unigram search);
- 0 is trivial: it only occurrs at the end of every message and therefore its MI is 0 bits (always true), so we conclude it is an 'end message' token; we will not look for it;
- any other pattern with 0 bits does not exist in the dataset;
- a pattern is a combination of n unigrams with 1 to 8 repetitions (1 repetition is the single occurrence of a token);
- if n repetitions of a token does not reduce or only slightly reduces MI w.r.t. n-1 repetitions of that token, then it is a loopy pattern on n-1 repetitions of the token (it carries the same categorical information);
- for any pattern, we can multiply each component of the pattern to explore the combinations of n x n lengths and repetitions. <br>

We predict that:
- some shorter patterns will have high MI and reflect a category or a set of categories;
- the lengthier the pattern, the more message-specific they will get; therefore, the longest patterns will have low or no MI; for specific semantics, short to medium length patterns will be most informative.

This does not consider:
- repetitions of two or more letters;
- repetitions of patterns that include repetition;
- that a tokenizer may be better suited for the task;
- that the ultimate goal is to reconstruct a grammar automatically.

About patterns:
- high numbers of reps likely not more informative than low nbs: (e.g. 12, 12 12; 12 1, 12 1 12 1)
- look for conjunctions and disjunctions that increase MI for possible "synonyms"
- and signals such as 12 x x x 1

In [ ]:
elems = ['/1/', '/2/', '/4/', '/5/', '/6/', '/9/', '/10/', '/11/', '/12/', '/13/', '/14/', '/15/']

In [ ]:
els = ['(/1/)', '(/2/)', '(/4/)', '(/5/)', '(/6/)', '(/9/)',
'(/10/)', '(/11/)', '(/12/)', '(/13/)', '(/14/)', '(/15/)']
multiples = ['{1}', '{2}', '{3}', '{4}', '{5}', '{6}', '{7}', '{8}']

In [ ]:
from itertools import permutations, product

def generate_ordered_ngram_patterns(elements, multiples, n):
  """
  Generates ordered n-gram patterns with multiplier.

  elements  - list of tokens
  multiples - list of numbers of possible repetitions
  n         - n-gram size

  Yields a string of tokens with multipliers (regex-style)
  """
  counter = 0
  for tokens in permutations(elements, n):
    for multi_tuple in product(multiples, repeat=n):  # add multiplier to each token
      pattern = ''.join(token + mult for token, mult in zip(tokens, multi_tuple))
      yield pattern
      counter += 1
      if counter % 10000 == 0:
        print(f"generated {counter} patterns")
  print(f"generated {counter} patterns")

In [ ]:
results = {}
for pattern in generate_ordered_ngram_patterns(els, multiples, 2):
  mi = test_feature(feature_f=(lambda signal, pat=pattern: re.search(pat,signal) is not None),
                    signals=signals, categories=categories)
  if mi != 0.0:
    results[pattern] = round(mi, 6)

KeyboardInterrupt: 

In [ ]:
for reg, score in results.items():
  print(f"{reg},{score}")

### Unique msg / cat

In [ ]:
df = df.sort_values("cat")
df_uniques = df.drop_duplicates().groupby("cat").size()
df_counts = df.groupby("cat").size()

df_unique_values = df.groupby("cat").agg(lambda x: list(x.unique()))
df_unique_values = df_unique_values.iloc[:, 0]

# Basically how robust a description is for a given cat
comparison_df = pd.DataFrame({"df_uniques_counts": df_uniques,
                              "df_counts": df_counts,
                              "unique_values": df_unique_values
                              })

comparison_df.sort_values("df_uniques_counts")

,df_uniques_counts,df_counts,unique_values
cat,,,
1 1 1 1 1,1,277,[12 12 12 12 12 12 5 0]
0 0 0 1 1,1,237,[12 1 0]
1 1 0 1 1,1,264,[12 12 12 0]
1 0 0 0 0,1,270,[1 0]
0 1 0 0 1,1,235,[1 4 0]
0 0 0 0 1,2,254,"[12 4 0, 4 4 4 4 4 4 4 0]"
1 1 1 0 1,2,251,"[9 9 9 9 9 9 9 1 0, 9 9 9 9 9 9 1 0]"
1 0 1 1 1,2,252,"[12 9 9 9 9 9 9 0, 12 9 9 9 4 4 0]"
1 1 0 0 1,3,233,"[1 4 0, 12 12 1 12 1 1 1 0, 12 12 12 12 12 12 ..."


In [ ]:
comparison_df.sort_values("df_uniques_counts").to_csv("comparison.csv", index=True)
from google.colab import files
files.download("comparison.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Archives

In [ ]:
def permutation(expression):
  mi = test_feature(feature_f=(lambda signal: (re.search(expression, signal) is not None)), signals=signals, categories=categories)
  if mi/math.log(2) != 0.0:
    print(expression, mi/math.log(2))

In [ ]:
from itertools import permutations

expressions = ['/1/', '/2/', '/4/', '/5/', '/6/', '/8/', '/9/', '/10/', '/11/', '/12/', '/13/', '/14/', '/15/']

n = 3
n_grams = [''.join(combo) for combo in permutations(expressions, n)]

for gram in n_grams:
  permutation(gram)
"""
for exp in expressions:
  for expre in expressions:
    if exp != expre:
      permutation(exp + expre)"""

In [ ]:
def permutation2(expression):
  regex = re.compile(expression)
  mi = test_feature(
      feature_f=(lambda signal: (re.search(expression, signal) is not None)),
      signals=signals, categories=categories
  )
  if mi/math.log(2) != 0.0:
    print(expression, mi/math.log(2))

In [ ]:
from itertools import permutations
from multiprocessing import Pool

expressions = ['/1/', '/2/', '/4/', '/5/', '/6/', '/8/', '/9/', '/10/', '/11/', '/12/', '/13/', '/14/', '/15/']

n = 5
n_grams = [''.join(combo) for combo in permutations(expressions, n)]

with Pool() as pool:
  pool.map(permutation, (''.join(gram) for gram in permutations(expressions, n)))

In [ ]:
"""
els = ['(/1/)', '(/2/)', '(/4/)', '(/5/)', '(/6/)', '(/7/)', '(/9/)',
'(/10/)', '(/11/)', '(/12/)', '(/13/)', '(/14/)', '(/15/)']
multiples = '{0}', '{1}', '{2}', '{3}', '{4}', '{5}', '{6}', '{7}', '{8}'
unigr_out = []
bigr_out_first = []
bigr_out_second = []

for el in els:
  for ele in els:
    for mul in multiples:
      feat_unigr = el + mul
      if el != ele:
        feat_first = el + mul + ele
        feat_second = el + ele + mul
      mi = test_feature(feature_f=(lambda signal: (re.search(feat_unigr, signal) is not None)), signals=signals, categories=categories)
      if mi/math.log(2) != 0.0 and mul != '{0}' and (feat_unigr, mi/math.log(2)) not in unigr_out:
        unigr_out.append((feat_unigr, mi/math.log(2)))
      mi = test_feature(feature_f=(lambda signal: (re.search(feat_first, signal) is not None)), signals=signals, categories=categories)
      if mi/math.log(2) != 0.0 and mul != '{0}':
        bigr_out_first.append((feat_first, mi/math.log(2)))
      mi = test_feature(feature_f=(lambda signal: (re.search(feat_second, signal) is not None)), signals=signals, categories=categories)
      if mi/math.log(2) != 0.0 and mul != '{0}':
        bigr_out_second.append((feat_second, mi/math.log(2)))

# next step: optimization then trigrams """

In [ ]:
""" Deprecated (it can be simpler)

from itertools import product, permutations

unigr_out = {}
bigr_out_first = {}
bigr_out_second = {}

# ---Unigram generation---
# A unigram is an element possibly multiplied by a factor in multiples
for el, mul in product(els, multiples):
  # Skip product of 0
  if mul == '{0}':
    continue
  feat_unigr = el + mul
  mi = test_feature(feature_f=(lambda signal, pattern=feat_unigr: re.search(pattern, signal) is not None),
                    signals=signals, categories=categories)
  value_uni = mi/log2
  if value_uni != 0.0:
    unigr_out[feat_unigr] = round(value_uni, 6)

# ---Bigram generation---
# A bigram are two order-sensitive elements combined with a multiple of either
for el, ele, mul in product(els, els, multiples):
  # Only consider bigrams with two different elements
  if el == ele or mul == '{0}':
    continue
  feat_bi_first = el + mul + ele
  feat_bi_second = el + ele + mul

  mi_bi_first = test_feature(feature_f=(lambda signal, pattern=feat_bi_first: re.search(pattern, signal) is not None),
                          signals=signals, categories=categories)
  value_bi_first = mi_bi_first/log2
  if value_bi_first != 0.0:
    bigr_out_first[feat_bi_first] = round(value_bi_first, 6)

  mi_bi_second = test_feature(feature_f=(lambda signal, pattern=feat_bi_second: re.search(pattern, signal) is not None),
                          signals=signals, categories=categories)
  value_bi_second = mi_bi_second/log2
  if value_bi_second != 0.0:
    bigr_out_second[feat_bi_second] = round(value_bi_second, 6) """

In [ ]:
""" Deprecated! (it gets even simpler that that!)

from itertools import permutations, combinations, product

def generate_ordered_ngram_patterns(elements, multiples, n, n_multiples):

  Generate ordered n-gram patterns with multiples of one or several elements.
  Multiples are inserted into available slots.

  els           - list of possible tokens
  multiples     - list of multiplications for regex search
  n             - number of tokens to consider for the n-gram
  n_multiples   - number of tokens that will be multiplied

  Yields string patterns (strings with tokens and inserted multiples regex-style).

  for tokens in permutations(els, n):
    slots = range(1, n+1)
    for pos_indices in combinations(slots, n_multiples):  # num_multiples slots
      for multi_tuple in product(multiples, repeat=n_multiples):
        #if any(mul == '{0}' for mul in multi_tuple):
        #  continue
        pattern = ''
        for i, token in enumerate(tokens):  # iterate over each slot index
          pattern += token
          if (i+1) in pos_indices:  # add corresponding multiple to selected slot
            index = pos_indices.index(i+1)
            pattern += multi_tuple[index]
        yield pattern """